In [6]:
from google.colab import files
uploaded = files.upload()

Saving gut.txt to gut (2).txt


It is defunct and does not work. I've started it too late.

In [2]:
text = open('gut.txt', 'r', encoding = 'utf-8').read()[2:]
len(text)

3227577

Проверить работу через два for в первом случае

In [3]:
import string
import re

def preprocess_text(text):
    new_text = text.lower()
    #new_text = re.sub(r'[!?"<>:;,—–«»”“’@]','',new_text)
    new_text = new_text.translate(str.maketrans('', '', ',—*)[-];:/@!“?’#%=”‘('))
    new_text = re.sub(r"\s+",' ',new_text)
    del text
    return new_text

text = preprocess_text(text)
len(text)
#assert len(text) == 3141169

3130291

In [4]:
text = text.split('.')
text = [x.strip() for x in text]

In [5]:
def flatten(list):
  for i in list:
    for j in i:
      yield j

In [ ]:
from collections import Counter
from sklearn.base import TransformerMixin


class BPE(TransformerMixin):
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        for sentence in text:
          text.append(list(sentence))
        #text = [list(sentence) for sentence in text]
        #text = list(flatten(text))
        #self.itos = list(set(list(flatten(text))))
        #for i, item in enumerate(self.itos):
          #self.stoi[item] = i
        for sent in text:
            for char in sent:
                if char not in self.itos:
                    self.itos.append(char)
                    self.stoi[char] = self.itos.index(char)
        
        text = [[self.stoi[sym] for sym in sent] for sent in text]
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigram_frq = Counter(zip(list(flatten(text)), list(flatten(text))[1:]))
            new_token = bigram_frq.most_common(1)[0][0]
            # most common bigram in the text
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            
            # find occurences of the new_token in the text and replace them with new_id
            for sent in text:
              indices = [i for i, x in enumerate(sent) if x == new_token[0]]
              remove = []

              for index in indices:
                if index!=len(sent)-1:
                  if sent[ind+1] == new_token[1]:
                    remove.append(ind+1)
                    sent[index] = new_id
              
              sent = [i for j, i in enumerate(sent) if j not in remove]
            
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """
        # TODO tokenize text by symbols using self.stoi
        #text = [list(sentence) for sentence in text]
        #text = list(flatten(text))
        text = [[self.stoi[sym] for sym in sent] for sent in text]
        
        
        
        
        for sent in text:
          for el in range(len(self.itos)):
            indices = [i for i, x in enumerate(sent) if x == el]
  
            new_itos = [k for k in self.itos if type(k)!=str and k[0] == el]
          #print(el, needed_itos)
            for ix in indices:
              if ix<len(sent)-1:
                for item in new_itos:
                  if sent[ix+1] == item[1]:
                    sent = sent[:ix]+[self.stoi[item]]+sent[ix+2:] 
                  
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        # and credits go to StackOverflow
        def recursive_dec(token): #token int or tuple
          if type(token) == int:
            sign = self.itos[token]
            if type(sign) == str:
              return sign 
            else:
              return recursive_dec(sign)
          elif type(token) == tuple:
            return recursive_dec(token[0]) + recursive_token_lookup(token[1]) 
        return recursive_dec(tok)
        result = []# TODO
        return recursive_dec(tok)
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [0]:
import numpy as np
        
    
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = {}
        
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = []
        for token in range(self.vocab_size):
            result.append(self.get_proba(a, b, token, tau))
        return np.array(result)
      
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        
        """
        probability3 = []
        for token in range(self.vocab_size):
            probability3.append((self.proba[(a, b, c)]+self.delta)** (1/tau))
        return ((self.proba[(a, b, c)]+self.delta)** (1/tau)) / sum(probability3)
      
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        
        trigrams = []
        for sent in text:
            all = [start_token] + sent + [end_token]
            for i in range(len(all) - 2):
                trigrams.append((all[i], all[i + 1], all[i + 2]))
        self.proba = Counter(trigrams)
        
        return self

In [0]:
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
import numpy as np
from itertools import tee, islice


start_token = vocab_size
end_token = vocab_size + 1

bpe.itos.append('@')
bpe.itos.append(' ')

bpe.stoi['@'] = start_token
bpe.stoi['.'] = end_token

def ngrams(lst, n):
  tlst = lst
  while True:
    a, b = tee(tlst)
    l = tuple(islice(a, n))
    if len(l) == n:
      yield l
      next(b)
      tlst = b
    else:
      break
      
class LM:
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = [] # TODO create array for storing 3-gram counters
        
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        
        text = [[start_token]+sent+[end_token] for sent in text]
        text = list(flatten(text))
        self.proba = Counter(ngrams(text, 3))
        return self
      
    
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        
        #вероятност триграммы абс
        """
        counter_result = self.proba
        count_current_3gram = counter_result[(a,  b, c)]
        count_all_3grams = 0
        for count in counter_result:
          if count[0]==a and count[1]==b:
            count_all_3grams += counter_result[count]
        result = ((count_current_3gram + self.delta) ** (1/tau))/((count_all_3grams + self.delta) ** (1/tau))
        # TODO approximate probability by counters
        return result
      
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        ## вектор длины 100 (размер вокаба) в котором хранятся вероятности всех триграмм , начинающихся с а b
        a: first token id
        b: second token id
        tau: temperature
        """
        vector = [] #[0] * len(self.vocab)
        for item in range(self.vocab_size):
          vector.append(self.get_proba(a, b, item, tau=1))
        result = vector
        #print(result)
        return result
      
      
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """

    beam = [(input_seq, 1)]
    
    # TODO store in beam tuples of current sequences and their log probabilities
    
    for i in range(max_len-len(input_seq)): ###тут минус уже имеющееся наверное??????):
      #print ('BEAM', beam)
      candidates = []
      candidates_proba = [] #LM(vocab_size, 1).infer(a, b, tau=1)
    
      for item in beam:
          #print(item)
          snt = item[0]
          snt_proba = item[1]
          #print(type(snt))
          #print(snt[-1], snt_proba)
          x = snt[-2]
          y = snt[-1]
          #print(a,b)
          if y == end_token:
            continue
          else:    
            proba = lm.infer(x, y, tau) # probability vector of the next token
              #print(len(proba))
            sorted_probs = sorted([(e,i) for i,e in enumerate(proba)], reverse = True)
              #print(len(bpe.itos))
              #print(sorted_probs)
            best_k = [bpe.itos[ix[1]] for ix in sorted_probs[:k]]# top-k most probable tokens
              
            for token in best_k:
                candidates.append(item[0]+[bpe.stoi[token]])
                candidates_proba.append(sorted_probs[bpe.stoi[token]][0])
                # TODO update candidates' sequences and corresponding probabilities
                
      beam = [(candidates[i],candidates_proba[i]) for i in range(k)]
      
    return beam

In [40]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for item in result:
  print(bpe.decode(item[0]), item[1])

horse and  0.7665664392274777
horse ande 0.0007806175552214641
horse andr 2.891176130449867e-05
horse andi 2.891176130449867e-05
horse ands 2.891176130449867e-05
horse and  0.7665664392274777
horse ando 0.00034694113565398406
horse andl 2.891176130449867e-05
horse anda 2.891176130449867e-05
horse andu 2.891176130449867e-05
